In [1]:
import urllib, urllib.request as req, urllib.parse
import pprint
from bs4 import BeautifulSoup
import csv

REVIEW_URL = r"http://www.allocine.fr/films/"
user_agent_ = 'User-agent'
mozilla_ = 'Mozilla/5.0'

asins = []
proxy = req.ProxyHandler({'http': r'http://:@proxy-sgt.si.socgen:8080'})
auth = req.HTTPBasicAuthHandler()
opener = req.build_opener(proxy, auth, req.HTTPHandler)
req.install_opener(opener)
opener.addheaders = [(user_agent_, mozilla_)]

In [2]:
url = REVIEW_URL
response = opener.open(url)    
data = response.read()
soup = BeautifulSoup(data, "lxml")

html_dict = {}
html_pref = r'http://www.allocine.fr'

soup = soup.findAll('div', {'class' : 'filter-entity-section'})[0]
for genre in soup.findAll('li', {'class' : 'filter-entity-item'}):
        html = genre.findAll('a')[0]
        html_href = html_pref + html['href']
        
        number = genre.findAll('span')[0].text
        number = number[1:(len(number)-1)]
        
        html_dict[html_href] = int(number)
        
print(html_dict)

{'http://www.allocine.fr/films/genre-13010/': 748, 'http://www.allocine.fr/films/genre-13051/': 27, 'http://www.allocine.fr/films/genre-13012/': 2732, 'http://www.allocine.fr/films/genre-13040/': 11, 'http://www.allocine.fr/films/genre-13018/': 3997, 'http://www.allocine.fr/films/genre-13019/': 1282, 'http://www.allocine.fr/films/genre-13017/': 28547, 'http://www.allocine.fr/films/genre-13031/': 164, 'http://www.allocine.fr/films/genre-13015/': 1660, 'http://www.allocine.fr/films/genre-13009/': 4372, 'http://www.allocine.fr/films/genre-13002/': 7007, 'http://www.allocine.fr/films/genre-13033/': 78, 'http://www.allocine.fr/films/genre-13014/': 1485, 'http://www.allocine.fr/films/genre-13048/': 129, 'http://www.allocine.fr/films/genre-13047/': 50, 'http://www.allocine.fr/films/genre-13022/': 252, 'http://www.allocine.fr/films/genre-13027/': 1692, 'http://www.allocine.fr/films/genre-13005/': 14607, 'http://www.allocine.fr/films/genre-13026/': 2136, 'http://www.allocine.fr/films/genre-1301

In [ ]:
for key, value in html_dict.items():
    print(key, value)
    url = key + r'?page=%s'
    i = 1
    j = 0
    while j<value:
        url_page = url % i
        i +=1
        response = opener.open(url_page)    
        data = response.read()

        soup = BeautifulSoup(data, "lxml")

        for rawreview in soup.findAll('a', {'class' : 'meta-title-link'}):
            html = rawreview['href']
            asin = html[html.index("film="):]
            asin = asin.split('.')[0]
            asin = asin.split('=')[1]
            asins.append(asin)
            j +=1

In [3]:
myfile = open("../dataset/asins.csv", 'r')
wr = csv.reader(myfile)
for row in wr:
    asins = row
    break

In [ ]:
REVIEW_URL = r"http://www.allocine.fr/film/fichefilm-%s/critiques/spectateurs/?page=%s"
user_agent_ = 'User-agent'
mozilla_ = 'Mozilla/5.0'
output_data = '../dataset/data_allocine_v5.csv'
output_stats = '../dataset/rating_allocine_v5.csv'

asins = set(asins)
rating_dict = {}

#last_movie = "19848"
#asins = asins[asins.index(last_movie):]

# Parse a url for product reviews
# return tuple of (review text, rating)
def parse_reviews(url):
    response = opener.open(url)    
    data = response.read()
    
    soup = BeautifulSoup(data, "lxml")
    reviews = []
    
    for rawreview in soup.findAll('div', {'itemprop' : 'review'}):
        rawrating = rawreview.findAll('span', class_ = 'stareval-note')
        rating = rawrating[0]['content'].replace(",", ".")
        
        rawtext = rawreview.findAll('p', {'itemprop' : 'description'})
        text = rawtext[0].text
        
        # cleaning text from breaklines ('\n') at beginning and end of text
        text = text.lstrip('\n \n').rstrip('\n \n').replace(';', ',').replace('\n', '')
        reviews.append((text, rating))
        
    return reviews

def store_reviews(product_id, reviews):
    added_count = 0
    with open(output_data, 'a', encoding='utf-8') as out:
        csv_out = csv.writer(out, delimiter = ';')
        for (data, rating) in reviews:
            csv_out.writerow((data, rating))
            if rating in rating_dict:
                rating_dict[rating] += 1
            else:
                rating_dict[rating] = 1
                
            added_count += 1
            
    return added_count

# Download all reviews for a giving product id (ASIN)
def download_reviews(product_id):
    i = 1
    added = 0
    print("download movie", product_id)
    
    # Retrieve reviews count:
    url =  REVIEW_URL % (product_id, i)
    response = opener.open(url)    
    data = response.read()
    soup = BeautifulSoup(data, "lxml")
    try:
        reviews_count = int(soup.findAll('span', {'itemprop' : 'reviewCount'})[0].text)
        print("There are %s reviews" % reviews_count)

        while added < reviews_count:
            url = REVIEW_URL % (product_id, i)
            print("URL", url)
            reviews = parse_reviews(url)
            added += store_reviews(product_id, reviews)
            i += 1
            if added == 0:
                break
    
    except IndexError:
        pass


def main():
    open(output_data, 'w').close()
    open(output_stats , 'w').close()
    for asin in asins:
        download_reviews(asin)
    
    with open(output_stats, 'w') as out:
        csv_out = csv.writer(out)
        for key, value in sorted(rating_dict.items()):
            print("%s : %s" %(key, value))
            csv_out.writerow((key, value))

if __name__=="__main__":
    main()

download movie 52720
There are 2 reviews
URL http://www.allocine.fr/film/fichefilm-52720/critiques/spectateurs/?page=1
download movie 183402
There are 4 reviews
URL http://www.allocine.fr/film/fichefilm-183402/critiques/spectateurs/?page=1
download movie 180172
download movie 125174
There are 129 reviews
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=1
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=2
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=3
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=4
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=5
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=6
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=7
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spectateurs/?page=8
URL http://www.allocine.fr/film/fichefilm-125174/critiques/spe